In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
mutations_df = pd.read_csv('../data/WES_variants.csv')
mutations_df.head()

,SAMPLE,COSMIC_ID,Cancer Type,Gene,Transcript,cDNA,AA,Classification,Gene_list,Recurrence Filter,Subs,Truncating,inframe
0,KARPAS-45,907272,ALL,A1BG,ENST00000263100,c.842A>G,p.Y281C,missense,NaN,NaN,NaN,NaN,NaN
1,Jurkat,998184,ALL,A1BG,ENST00000263100,c.589G>A,p.A197T,missense,NaN,NaN,NaN,NaN,NaN
2,KARPAS-45,907272,ALL,A1BG,ENST00000263100,c.571G>A,p.A191T,missense,NaN,NaN,2.0,NaN,NaN
3,HT-115,907289,COAD/READ,A1BG,ENST00000263100,c.1456G>A,p.D486N,missense,NaN,NaN,NaN,NaN,NaN
4,KM12,905989,COAD/READ,A1BG,ENST00000263100,c.770C>A,p.T257N,missense,NaN,NaN,NaN,NaN,NaN


In [26]:
#mutations_df.count()

grouped_mut_df = mutations_df.groupby('COSMIC_ID')

grouped_mut_df.count()



,SAMPLE,Cancer Type,Gene,Transcript,cDNA,AA,Classification,Gene_list,Recurrence Filter,Subs,Truncating,inframe
COSMIC_ID,,,,,,,,,,,,
683665,193,193,193,193,193,193,193,8,10,15,19,0
683667,119,119,119,119,119,119,119,7,2,17,7,0
684052,105,0,105,105,105,105,105,6,5,13,10,0
684055,150,0,150,150,150,150,150,13,7,10,16,0
684057,159,0,159,159,159,159,159,7,6,15,13,0
684059,180,0,180,180,180,180,180,13,7,18,18,0
684062,250,0,250,250,250,250,250,15,10,32,20,1
684072,140,0,140,140,140,140,140,6,8,17,21,0
684681,237,237,237,237,237,237,237,8,17,36,22,0


In [3]:
dose_response_df = pd.read_csv('../data/v17.3_fitted_dose_response.csv', index_col='IC50_RESULTS_ID')
dose_response_df.head()

,DATASET_VERSION,COSMIC_ID,CELL_LINE_NAME,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,MAX_CONC_MICROMOLAR,MIN_CONC_MICROMOLAR,LN_IC50,AUC,RMSE,Z_SCORE
IC50_RESULTS_ID,,,,,,,,,,,,
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.982610,0.021678,-0.015505
1482,17.3,684055,ES3,1,Erlotinib,EGFR,2.0,0.007812,3.376592,0.985169,0.029915,0.779999
1791,17.3,684057,ES5,1,Erlotinib,EGFR,2.0,0.007812,3.614664,0.983207,0.031201,0.985170
2177,17.3,684059,ES7,1,Erlotinib,EGFR,2.0,0.007812,3.223394,0.984574,0.093857,0.647971
2754,17.3,684062,EW-11,1,Erlotinib,EGFR,2.0,0.007812,2.486405,0.946034,0.087280,0.012832


In [5]:
joined_df = dose_response_df.join(mutations_df.set_index('COSMIC_ID'), on='COSMIC_ID')
joined_df.head()

,DATASET_VERSION,COSMIC_ID,CELL_LINE_NAME,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,MAX_CONC_MICROMOLAR,MIN_CONC_MICROMOLAR,LN_IC50,AUC,...,Gene,Transcript,cDNA,AA,Classification,Gene_list,Recurrence Filter,Subs,Truncating,inframe
IC50_RESULTS_ID,,,,,,,,,,,,,,,,,,,,,
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.98261,...,ABAT,ENST00000396600,c.26G>A,p.R9H,missense,NaN,NaN,NaN,NaN,NaN
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.98261,...,ABCA8,ENST00000269080,c.3309+5G>T,p.?,ess_splice,NaN,Yes,NaN,28.0,NaN
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.98261,...,ABCG5,ENST00000260645,c.1436C>T,p.T479I,missense,NaN,NaN,NaN,NaN,NaN
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.98261,...,ACAP1,ENST00000158762,c.2206G>A,p.D736N,missense,NaN,NaN,NaN,NaN,NaN
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.98261,...,ADCY4,ENST00000310677,c.2804C>G,p.T935R,missense,NaN,NaN,NaN,NaN,NaN


In [16]:
mutation_dose_df = joined_df[['COSMIC_ID', 'LN_IC50', 'cDNA','AA','DRUG_ID']]

In [17]:
mutation_dose_df.head()

,COSMIC_ID,LN_IC50,cDNA,AA,DRUG_ID
IC50_RESULTS_ID,,,,,
1,683665,2.453524,c.26G>A,p.R9H,1
1,683665,2.453524,c.3309+5G>T,p.?,1
1,683665,2.453524,c.1436C>T,p.T479I,1
1,683665,2.453524,c.2206G>A,p.D736N,1
1,683665,2.453524,c.2804C>G,p.T935R,1


In [18]:
mutation_dose_df.reset_index(drop=True, inplace=True)

In [21]:
mutation_dose_df.head()

,COSMIC_ID,LN_IC50,cDNA,AA,DRUG_ID
0,683665,2.453524,c.26G>A,p.R9H,1
1,683665,2.453524,c.3309+5G>T,p.?,1
2,683665,2.453524,c.1436C>T,p.T479I,1
3,683665,2.453524,c.2206G>A,p.D736N,1
4,683665,2.453524,c.2804C>G,p.T935R,1


In [20]:
mutation_dose_df.to_csv('../data/wes_dose.csv')

In [23]:
mutation_dose_df.count()

COSMIC_ID    102663661
LN_IC50      102663661
cDNA         102652824
AA           102652824
DRUG_ID      102663661
dtype: int64